In [1]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.21"
    "NBitcoin -Version 3.0.0.33"
    "FSharp.Data -Version 2.4.6"
    "Newtonsoft.Json -Version 10.0.3"    
  ]    
#load "Paket.Generated.Refs.fsx"

open System
open NBitcoin
open NBitcoin.Crypto  


In [5]:
type Wallet =
    static member PrivateKey: Key = 
        (new Key())
    static member PublicKey (k:Key) = 
        if (k=null) then
          (new Key()).PubKey
        else
          k.PubKey 
          
type Wallet with
     static member BitcoinSecretMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.Main)
            else
                k.GetBitcoinSecret(Network.Main)
     static member BitcoinSecretTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.TestNet)
            else
                k.GetBitcoinSecret(Network.TestNet)    
     static member WifMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.Main)
            else
                k.GetWif(Network.Main)
     static member WifTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.TestNet)
            else
                k.GetWif(Network.TestNet)
type Wallet with
     static member BitCoinAddressMain (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.Main) 
     static member BitCoinAddressTest  (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.TestNet) 
     static member PublicKeyHash (k:Key) = 
        Wallet.PublicKey(k).Hash 
     static member BitCoinAddressMain2 (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.Main) 
     static member BitCoinAddressTest2  (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.TestNet) 
        
type Script = 
   static member PublicKeyHash (Pk:string) = 
       new KeyId(Pk) 
   static member Main  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.Main).ScriptPubKey
   static member Test  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.TestNet).ScriptPubKey 
   static member Main  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.Main).ScriptPubKey 
   static member Test  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.TestNet).ScriptPubKey 
   static member BitCoinAddressMain (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.Main)
   static member BitCoinAddressTest  (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.TestNet)

In [6]:
open FSharp.Data
open FSharp.Data.JsonExtensions

type secretJson = JsonProvider<"""{
  "secret": ""
}""">
      
let sj = secretJson.GetSample()
//printfn "%A" sj 
  
let json_uri = @"/home/nbuser/library/secret.json"
let wwif = secretJson.Load(json_uri).JsonValue?secret.AsString()  

let network = Network.TestNet

let sender_bitcoinPrivateKey  = Key.Parse(wwif, Network.TestNet) 
let sender_address = Script.BitCoinAddressTest sender_bitcoinPrivateKey //Wallet.PublicKeyHash bitcoinPrivateKey

printfn "%A" (network)
printfn "sender %A" (sender_address)
printfn "sender %A" (Script.Test sender_bitcoinPrivateKey)

TestNet
sender mnnaUEKBKJVFjCqVnVPmwefk6CwuVsyRRj
sender OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG


In [8]:
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System.Collections.Generic 

let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)
let transactionId = uint256.Parse("d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b") 
let transactionResponse:GetTransactionResponse = client.GetTransaction(transactionId).Result 

printfn "%A" transactionResponse.TransactionId // d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b
printfn "%A" transactionResponse.Block.Confirmations  // 200
 

d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b
1481


Da Dove?

In [9]:
let receivedCoins = transactionResponse.ReceivedCoins;
let mutable outPointToSpend:OutPoint = null 
for coin in receivedCoins do
    if (coin.TxOut.ScriptPubKey =  sender_bitcoinPrivateKey.ScriptPubKey) then
        outPointToSpend <- coin.Outpoint 
    
if(outPointToSpend = null) then
    failwith "TxOut doesn't contain our ScriptPubKey" 

printfn "%A" outPointToSpend

printfn "Vogliamo spendere dal %A-mo outpoint" (outPointToSpend.N + 1u) //base zero 

let transaction = new Transaction() 
let mutable fromWhere = new TxIn()
fromWhere.PrevOut <- outPointToSpend
transaction.Inputs.Add(fromWhere)

printfn "%A" transaction

d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b-0
Vogliamo spendere dal 1u-mo outpoint
{
  "hash": "3468c529ad2041e993a276478cb23341d8513e90319d60a8d70edccd1583ed04",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 0,
  "lock_time": 0,
  "size": 51,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 0
      },
      "scriptSig": ""
    }
  ],
  "out": []
}


Verso chi?  destination **mtYy84R9Ezc6UxWGfHgMj4TmxXFqSy7SPf**

In [10]:
let json_uri_dest = @"/home/nbuser/library/secret_dest.json"
let wwif_dest = secretJson.Load(json_uri_dest).JsonValue?secret.AsString()  


let dest_bitcoinPrivateKey  = Key.Parse(wwif_dest, Network.TestNet) 
let dest_address = Script.BitCoinAddressTest dest_bitcoinPrivateKey  
printfn "destination %A" (dest_address)
printfn "destination %A" (Script.Test dest_bitcoinPrivateKey)

destination mtYy84R9Ezc6UxWGfHgMj4TmxXFqSy7SPf
destination OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG


Quanto? **1 satoshi**

In [11]:
let mutable toWhereTxOut = new TxOut()
toWhereTxOut.Value <- new Money(0.00000001m, MoneyUnit.BTC) 
toWhereTxOut.ScriptPubKey <- dest_bitcoinPrivateKey.ScriptPubKey

//amount: 0.55M

// quanto voglio spendere
let wantSpend = new Money(0.1m, MoneyUnit.BTC);


printfn "%A" wantSpend

0.10000000


il fee può dipendere dal market price, facciamo 0.00007m

In [12]:
let minerFee = new Money(0.00007m, MoneyUnit.BTC);

// How much you want to get back as change
let myAmount:Money =  (receivedCoins.[(int) outPointToSpend.N].Amount):?>Money
let changeAmount = myAmount - wantSpend - minerFee;

let mutable changeBackTxOut = new TxOut ()
changeBackTxOut.Value <- new Money(changeAmount.ToDecimal(MoneyUnit.BTC), MoneyUnit.BTC) 
changeBackTxOut.ScriptPubKey <- sender_bitcoinPrivateKey.ScriptPubKey

printfn "changeAmount %A" (changeAmount)

changeAmount 0.43983000


aggiungo alla **transazione**

In [13]:
transaction.Outputs.Add(toWhereTxOut) 
transaction.Outputs.Add(changeBackTxOut) 

printfn "%A" transaction

{
  "hash": "f48a9c132616461ecf559857e56d2d4bdb8c723f98cd58be1810b32ab08acae1",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 119,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 0
      },
      "scriptSig": ""
    }
  ],
  "out": [
    {
      "value": "0.00000001",
      "scriptPubKey": "OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.43983000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    }
  ]
}


In [14]:
open System.Text

let message = "SF5128165575"
let bytes = Encoding.UTF8.GetBytes(message) 
printfn "%A %A" bytes bytes.Length



[|83uy; 70uy; 53uy; 49uy; 50uy; 56uy; 49uy; 54uy; 53uy; 53uy; 55uy; 53uy|] 12


//Data in OP_RETURN should have a maximum size of **83 bytes**

In [15]:
open System
open System.IO
open System.Security.Cryptography

let  GetStream (s:string) =
                let stream = new MemoryStream()
                let writer = new StreamWriter(stream)
                writer.Write(s)
                writer.Flush()
                stream.Position <- 0L
                stream
                
//Print the byte array in a readable format.
let ByteString (b:byte[]) = 
        let mutable i = 0
        let mutable s:string = ""
        for i = 0  to b.Length - 1 do
            s <- s + String.Format("{0:X2}", b.[i])
            if (i % 4 = 3) then
                s <- s+ " "
        s    
        
let ByteToHex bytes = 
    bytes 
    |> Array.map (fun (x : byte) -> System.String.Format("{0:X2}", x))
    |> String.concat System.String.Empty

In [16]:
let Sha256 (doc:string) = 
    let sdoc = GetStream doc
    let sh = new SHA256Managed()
    sh.ComputeHash(sdoc)

let hash = Sha256 message
printfn "%A" (hash)
printfn "%A" (ByteString(hash))
printfn "%A" (ByteToHex(hash))

[|75uy; 208uy; 55uy; 47uy; 197uy; 77uy; 18uy; 89uy; 0uy; 242uy; 6uy; 128uy; 6uy;
  169uy; 110uy; 170uy; 113uy; 106uy; 73uy; 25uy; 229uy; 216uy; 66uy; 245uy;
  197uy; 145uy; 194uy; 11uy; 33uy; 117uy; 98uy; 93uy|]
"4BD0372F C54D1259 00F20680 06A96EAA 716A4919 E5D842F5 C591C20B 2175625D "
"4BD0372FC54D125900F2068006A96EAA716A4919E5D842F5C591C20B2175625D"


In [17]:
let mutable messageTxOut = new TxOut ()
messageTxOut.Value <- Money.Zero
messageTxOut.ScriptPubKey <- TxNullDataTemplate.Instance.GenerateScriptPubKey(hash)

transaction.Outputs.Add(messageTxOut) 

In [18]:
printfn "%A" transaction


{
  "hash": "1c20811ce0afe57bfc11d3954b2f0b533071e651dbccee65c24bf8e138217c69",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 3,
  "lock_time": 0,
  "size": 162,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 0
      },
      "scriptSig": ""
    }
  ],
  "out": [
    {
      "value": "0.00000001",
      "scriptPubKey": "OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.43983000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.00000000",
      "scriptPubKey": "OP_RETURN 4bd0372fc54d125900f2068006a96eaa716a4919e5d842f5c591c20b2175625d"
    }
  ]
}


Sign your **transaction**

In [19]:
transaction.Inputs.[0].ScriptSig <- sender_address.ScriptPubKey 


In [20]:
printfn "%A" transaction

{
  "hash": "aaaf2e37817b6bbd7088b4fae7118ba6a32edb82104390c574ceccea016a50e9",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 3,
  "lock_time": 0,
  "size": 187,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 0
      },
      "scriptSig": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    }
  ],
  "out": [
    {
      "value": "0.00000001",
      "scriptPubKey": "OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.43983000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.00000000",
      "scriptPubKey": "OP_RETURN 4bd0372fc54d125900f2068006a96eaa716a4919e5d842f5c591c20b2175625d"
    }
  ]
}


In [18]:
//forniamo la private key per **firmare** la transazione
//transaction.Sign(sender_bitcoinPrivateKey, false)
//This construct is deprecated. Use Sign(Key,ICoin[]) instead)

In [19]:
let mutable broadcastResponse:BroadcastResponse = client.Broadcast(transaction).Result 


if not broadcastResponse.Success then
    printfn "ErrorCode: %A" (broadcastResponse.Error.ErrorCode) 
    printfn "Error message: %A" (broadcastResponse.Error.Reason) 
else
    printfn "Success!" 
    printfn "%A" (transaction.GetHash())

 

In [ ]:
[OP_RETURN](https://opreturn.org/)